<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Fitabase_marging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
fb_master = pd.read_csv("/content/drive/MyDrive/Iqram Sir/fitabse_new/Fitabase_combined_data_s1-s39.csv")
hr_aee = pd.read_csv("/content/drive/MyDrive/Iqram Sir/fitabse_new/Fitabase_HR_aEE_MAX_wo_radompoints.csv")
hr_recovery = pd.read_excel("/content/drive/MyDrive/Iqram Sir/fitabse_new/HR_Recovery.xlsx")
all_info = pd.read_excel("/content/drive/MyDrive/Iqram Sir/fitabse_new/Wearables-REDCap-CRF_001-039.xlsx")

all_info.rename(columns = {'Subject ID:':'Subject_ID'}, inplace = True)


In [21]:
fb_master.head()

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,Wound:.1,Wound Complication Notes/Clavien-Dindo Assessment Grade:.1,Hematological:.1,Hematological Complication Notes/Clavien-Dindo Assessment Grade:.1,Pain:.1,Pain Complication Notes/Clavien-Dindo Assessment Grade:.1,Complete?,Has this CRF been quality checked,Date CRF was last checked:,Complete?.1
0,002-JR,post,Light,10,1,69,762.275362,731.0,810.0,78.784007,...,,,,,,,,,,
1,002-JR,post,Light,10,10,64,752.296875,722.0,789.0,79.788063,...,,,,,,,,,,
2,002-JR,post,Light,10,100,73,687.876712,638.0,740.0,87.334350,...,,,,,,,,,,
3,002-JR,post,Light,10,101,78,585.282051,545.0,645.0,102.611992,...,,,,,,,,,,
4,002-JR,post,Light,10,102,66,603.772727,582.0,645.0,99.417782,...,,,,,,,,,,


#Adding New Columns Main dataset

In [19]:
for i in list(hr_aee.columns):
  if i != "Condition" and i!="Patient_Id":
    fb_master[i] = ''



for i in list(hr_recovery.columns):
  if i != "Subject" and i!="Condition" and i!="Segment" and i!="Max Heart Rate":
    fb_master[i+"(mean)"] = ''
    fb_master[i+"(stdev)"] = ''



for i in list(all_info.columns):
  if i != "Subject_ID" and i!="Record ID:":
    fb_master[i] = ''


<ipython-input-19-e5afbfb3740f>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fb_master[i] = ''
<ipython-input-19-e5afbfb3740f>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fb_master[i] = ''
<ipython-input-19-e5afbfb3740f>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fb_master[i] = ''
<

#Additional Functions

In [23]:
hr_aee.head()

,Patient_Id,Condition,HR_MAX(BPM),aEE_MAX(kJ/min)
0,001-JV,pre,126,70.39
1,001-JV,post,122,64.41
2,002-JR,pre,137,90.72
3,002-JR,post,145,91.98
4,003-ES,pre,130,41.08


In [24]:
hr_aee_back("002-JR","pre")

[('HR_MAX(BPM)', [137]), ('aEE_MAX(kJ/min)', [90.72])]

In [20]:
import statistics as stat


def hr_aee_back(p_id, condition):
  li = []
  for i in list(hr_aee.columns):
    if i!="Patient_Id" and i!="Condition":
      li.append((i,list(hr_aee.query("Patient_Id == '"+p_id+"' and 	Condition == '"+condition+"'")[i])))
  return li


def hr_recovery_back(p_id,condition):
  li = []
  for i in list(hr_recovery.columns):
    if i != "Subject" and i!="Condition" and i!="Segment" and i!="Max Heart Rate":
      li.append((i+"(mean)",stat.mean(list(hr_recovery.query("Subject == '"+p_id+"' and 	Condition == '"+condition+"'")['1_min_HR_Recovery']))))
      li.append((i+"(stdev)",stat.pstdev(list(hr_recovery.query("Subject == '"+p_id+"' and 	Condition == '"+condition+"'")['1_min_HR_Recovery']))))
  return li



def all_info_back(p_id):
  li = []
  for i in list(all_info.columns):
    if i!="Subject_ID" and i!="Record ID:":
      li.append((i,list(all_info.query("Subject_ID == '"+p_id+"'")[i])))
  return li

#Engine

---



In [25]:
fb_master.shape[0]

8314

In [ ]:
hr_aee_done = []
all_info_done = []
hr_recov = []

for i in tqdm(range(fb_master.shape[0])):
  try:
  #if fb_master["Subject"][i] in hr_subject:
    if (fb_master["Subject"][i],fb_master["Condition"][i]) not in hr_aee_done:
      temp_hr_aee = hr_aee_back(fb_master["Subject"][i],fb_master["Condition"][i])
      hr_aee_done.append((fb_master["Subject"][i],fb_master["Condition"][i]))

    for j in temp_hr_aee:
      fb_master[j[0]][i] = j[1][0]


  #if fb_master["Subject"][i] in hr_recovery_subject:
    if (fb_master["Subject"][i],fb_master["Condition"][i]) not in hr_recov:
      temp_hr_recovery = hr_recovery_back(fb_master["Subject"][i],fb_master["Condition"][i])
      hr_recov.append((fb_master["Subject"][i],fb_master["Condition"][i]))

    for j in temp_hr_recovery:
      fb_master[j[0]][i] = j[1]


  #if fb_master["Subject"][i] in all_info_subject:
    if fb_master["Subject"][i] not in all_info_done:
      temp_all_info = all_info_back(fb_master["Subject"][i])
      all_info_done.append(fb_master["Subject"][i])

    for j in temp_all_info:
      fb_master[j[0]][i] = j[1][0]

  except:
    print("Something went worng!!!. But Do not worry, Engine is doing its work")
  finally:
    continue




  0%|          | 0/8314 [00:00<?, ?it/s]

<ipython-input-26-411c76b99fe0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_master[j[0]][i] = j[1][0]
<ipython-input-26-411c76b99fe0>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_master[j[0]][i] = j[1]
<ipython-input-26-411c76b99fe0>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_master[j[0]][i] = j[1][0]
<ipython-input-26-411c76b99fe0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

Something went worng!!!. But Do not worry, Engine is doing its work
Something went worng!!!. But Do not worry, Engine is doing its work
Something went worng!!!. But Do not worry, Engine is doing its work
Something went worng!!!. But Do not worry, Engine is doing its work


In [ ]:
file_save_destination = '/content/drive/MyDrive/Iqram Sir/fitabse_new/marged/'

fb_master.to_csv(file_save_destination+"marged_version.csv",index=False)
